In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# If using a remote server it is neccessary to navigate to your workspace
import os 

os.chdir('/content/drive/MyDrive/BiasMitigation/LNTL')
print('Directory changed to...',  os.getcwd())

Directory changed to... /content/drive/MyDrive/BiasMitigation/LNTL


In [3]:
# -*- coding: utf-8 -*-

# file imports
! pip install import-ipynb 
import import_ipynb
import random
from option import get_option
from trainer_merger import *
from utils.utils_LNTL import save_option
from data_loaders.data_loader_cityscapes import *

# torch imports
import torch
from torch.backends import cudnn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data as data

# argparse jupyter workaround
from argparse import Namespace
# manually add the options becuase using juypter not command line:
# Must make it a Namespace class to be used as the argparser would be
root = '/content/drive/MyDrive/BiasMitigation/'

args = Namespace(exp_name = 'Exp5_Stage1', # Enter in format Exp1_Stage2
                 #n_class = 10,
                 #input_size = 28,
                 batch_size = 4,
                 momentum = 0.9,
                 optimiser = 'ADAM',  # accepts either ADAM or SGD
                 lr = 0.0001,  # try lr = 0.0001 for ADAM and lr = 0.001 SGD 
                 lr_decay_rate = 0.1,
                 lr_decay_period = 40,
                 weight_decay = 0.0001, #arbitrary value off net doubt we have time to tune. would like some paper to back up this ebing a good value
                 max_step = 100,
                 #depth = 20,
                 #color_var = 0.020,  
                 seed = 2,
                 ##############
                 checkpoint = None, 
                 log_step = 100 ,
                 save_step = 1, # save every epoch
                 dataset = 'Cityscapes',
                 data_dir = root + 'Datasets/Cityscapes/',
                 meta_dir = root + 'Datasets/Cityscapes/meta',
                 save_dir = root + 'LNTL/training_logs',
                 data_split = 'train',
                 use_pretrain = False, 
                 train_baseline = True,
                 train_greyscale = False,
                 #############
                 val_only_greyscale = True,
                 val_only_jitter = False,
                 #############
                 random_seed = None,
                 num_workers = 2,
                 cudnn_benchmark = True,
                 #############
                 cuda = True ,
                 debug = False, 
                 is_train = True,  
                 #############
                 network_type = 'Deeplab',  # Enter only Deeplab or SegNet
                 notes = 'Forcing the val dataloader to use greyscale images to check evaluation results, otherwise the same setup as Exp1_Stage1'
                 )


'''
parser.add_argument('-e', '--exp_name',   required=True,              help='experiment name')

parser.add_argument('--n_class',          default=10,     type=int,   help='number of classes')
parser.add_argument('--input_size',       default=28,     type=int,   help='input size')
parser.add_argument('--batch_size',       default=128,    type=int,   help='mini-batch size')
parser.add_argument('--momentum',         default=0.9,    type=float, help='sgd momentum')
parser.add_argument('--lr',               default=0.01,   type=float, help='initial learning rate')
parser.add_argument('--lr_decay_rate',    default=0.1,    type=float, help='lr decay rate')
parser.add_argument('--lr_decay_period',  default=40,     type=int,   help='lr decay period')
parser.add_argument('--weight_decay',     default=0.0005, type=float, help='sgd optimizer weight decay')
parser.add_argument('--max_step',         default=100,    type=int,   help='maximum step for training')
parser.add_argument('--depth',            default=20,     type=int,   help='depth of network')
parser.add_argument('--color_var',        default=0.03,   type=float, help='variance for color distribution')
parser.add_argument('--seed',             default=2,      type=int,   help='seed index')


parser.add_argument('--checkpoint',       default=None,               help='checkpoint to resume')
parser.add_argument('--log_step',         default=50,     type=int,   help='step for logging in iteration')
parser.add_argument('--save_step',        default=10,     type=int,   help='step for saving in epoch')
parser.add_argument('--data_dir',         default='./',               help='data directory')
parser.add_argument('--save_dir',         default='./',               help='save directory for checkpoint')
parser.add_argument('--data_split',       default='train',            help='data split to use')
parser.add_argument('--use_pretrain',     action='store_true',        help='whether it use pre-trained parameters if exists')
parser.add_argument('--train_baseline',   action='store_true',        help='whether it train baseline or unlearning')
parser.add_argument('--train_greyscale',  action='store_true',        help='whether to convert the images to greyscale to get a measure of no colour information')

parser.add_argument('--random_seed',                      type=int,   help='random seed')
parser.add_argument('--num_workers',      default=4,      type=int,   help='number of workers in data loader')
parser.add_argument('--cudnn_benchmark',  default=True,   type=bool,  help='cuDNN benchmark')


parser.add_argument('--cuda',             action='store_true',        help='enables cuda')
parser.add_argument('-d', '--debug',      action='store_true',        help='debug mode')
parser.add_argument('--is_train',         action='store_true',        help='whether it is training')
'''

def log_file_setup(option):
    # setting up training log files:
    if option.train_baseline:
        if option.train_greyscale:
            kind = 'baseline_greyscale'
            path = option.save_dir + '/' + option.network_type + '/' + option.dataset + '/baseline' + '/greyscale' + '/' + option.exp_name 
            if not os.path.exists(path):
                os.makedirs(path)
        else:
            kind = 'baseline_colour'
            # path = os.path.join(option.save_dir, '/' , option.network_type, '/', option.dataset, '/baseline', '/colour', '/', option.exp_name )
            path = option.save_dir + '/' + option.network_type + '/' + option.dataset + '/baseline' + '/colour' + '/' + option.exp_name
            if not os.path.exists(path):
                os.makedirs(path)
    else:
        kind = 'LNTL'
        #path = os.path.join(option.save_dir, '/' , option.network_type, '/', option.dataset, '/LNTL', '/', option.exp_name )
        path = option.save_dir + '/' + option.network_type + '/' + option.dataset + '/LNTL' + '/' + option.exp_name
        if not os.path.exists(path):
            os.makedirs(path)
    
    return path


def backend_setting(option):
    # log_dir = os.path.join(option.save_dir, option.exp_name)
    # if not os.path.exists(log_dir):
    #     os.makedirs(log_dir)

    if option.random_seed is None:
        option.random_seed = random.randint(1,10000)
    torch.manual_seed(option.random_seed)

    if torch.cuda.is_available() and not option.cuda:
        print('WARNING: GPU is available, but we are not using it')

    if not torch.cuda.is_available() and option.cuda:
        option.cuda = False
        print('Warning: Youve asked for CUDA but no GPU is available...setting CUDA to False')

    if option.cuda:
        os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
        #os.environ['CUDA_VISIBLE_DEVICES'] = ','.join([str(i) for i in option.gpu_ids])
        torch.cuda.manual_seed_all(option.random_seed)
        cudnn.benchmark = option.cudnn_benchmark
    
    if option.train_baseline:
        option.is_train = True


def main():
    #option = get_option()
    option = args
    #debug:
    #print(option)
    
    path = log_file_setup(option)
    backend_setting(option)
    trainer = Trainer(option, path)

    train_dataset = DatasetTrain(option)
    val_dataset = DatasetVal(option)


    train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                                batch_size=option.batch_size,
                                                shuffle=True,
                                                num_workers=option.num_workers)

    val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
                                                batch_size=option.batch_size,
                                                shuffle=False,
                                                num_workers=option.num_workers)

    if option.is_train:   
        save_option(option, path)
        trainer.train(train_loader = train_loader, val_loader = val_loader)
    else:
        trainer._validate(trainval_loader) # I dont think we need this

        pass

if __name__ == '__main__': main()

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=db99368a34493a356190139527ca0afea200b32715cb0d329fad93d288873a17
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
importing Jupyter notebook from option.ipynb
importing Jupyter notebook from trainer_merger.ipynb
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Changed the local path to.... /content/drive/MyDrive/BiasMitigation/LNTL
importing Jupyter notebook from /content/drive/My Drive/BiasMitigation/LNTL/models/Deeplab/deeplabv3.ipynb
importing Jupyter notebook from /content/drive/My Drive/BiasMitigation/LNTL/models/Deeplab/resnet.ipynb
importing Jupyter notebook from /content/drive/My Drive/BiasMitigation/LNTL/models/Deeplab/aspp.ipynb
importing Jupyter notebook from /content/drive/My Drive/BiasMitigation/LNTL/models/SegN

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3487: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details

Training epoch loss:.... 1.6879531


[Exp5_Stage1] INFO: [VAL] cls loss : 1.167123 (epoch 0.00)
[Exp5_Stage1] INFO: [VAL] cls loss : 0.922068 (epoch 0.80)


Validation epoch loss:.... 1.2554451


...

Validation epoch loss:.... 0.52798694
Checkpoint saved. Epoch : 98


[Exp5_Stage1] INFO: [TRAIN] cls loss : 1.879739 (epoch 99.00)
[Exp5_Stage1] INFO: [TRAIN] cls loss : 0.274726 (epoch 99.13)
[Exp5_Stage1] INFO: [TRAIN] cls loss : 0.767326 (epoch 99.26)
[Exp5_Stage1] INFO: [TRAIN] cls loss : 0.582669 (epoch 99.40)
[Exp5_Stage1] INFO: [TRAIN] cls loss : 0.197667 (epoch 99.53)
[Exp5_Stage1] INFO: [TRAIN] cls loss : 0.392506 (epoch 99.67)
[Exp5_Stage1] INFO: [TRAIN] cls loss : 0.485256 (epoch 99.80)
[Exp5_Stage1] INFO: [TRAIN] cls loss : 0.455411 (epoch 99.94)


Training epoch loss:.... 0.53907496


[Exp5_Stage1] INFO: [VAL] cls loss : 0.620029 (epoch 99.00)
[Exp5_Stage1] INFO: [VAL] cls loss : 0.382591 (epoch 99.80)


Validation epoch loss:.... 0.5319502
Checkpoint saved. Epoch : 99


In [ ]:
# # check the pickle file to see contents

# import pandas as pd

# lister = pd.read_pickle(r'/content/drive/MyDrive/BiasMitigation/LNTL/training_logs/Deeplab/Cityscapes/baseline/colour/Exp1_Stage1/epoch_losses_train.pkl')
# print(lister)

In [ ]:
# lister = pd.read_pickle(r'/content/drive/MyDrive/Deeplabv3Flat/training_logs/model_1/epoch_losses_train.pkl')
# print(lister)